<a href="https://colab.research.google.com/github/ranjithsrajan/PyLab/blob/main/MCP_rerank_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context Engineering for your MCP Client
## Dynamic MCP Selection: Using Contextual’s Reranker to Pick the Right Tools for Your Task
##### Written by: Qile Jiang
##### Last updated on: 7.31.25
##### Reference blog: https://contextual.ai/blog/context-engineering-for-your-mcp-client

Thousands of MCP servers exist and many are updated daily, making server selection difficult for LLMs.
- Current approaches require manually downloading and configuring servers, limiting flexibility.
- When multiple servers are pre-configured, LLMs get overwhelmed and confused about which server to use for specific tasks.

This workflow enables dynamic server selection from a live [PulseMCP](http://pulsemcp.com) directory of 5000+ servers.
- A user query goes to an LLM that decides whether to use MCP servers, then the LLM generates instructions for ranking relevant servers.
- [Contextual AI](https://contextual.ai)'s [reranker](https://contextual.ai/blog/introducing-instruction-following-reranker/) prioritizes servers based on these instructions, allowing real-time server discovery rather than being limited to manually downloaded servers.

In [ ]:
# Install required packages
!pip install requests openai openai mcp-use python-dotenv contextual-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.2 MB/s eta 0:00:00


In [ ]:
# Import libraries
import asyncio
import json
import tempfile
import os
import requests
from google.colab import userdata
from openai import OpenAI
from mcp_use import MCPAgent, MCPClient
from contextual import ContextualAI

## Step (1) We'll start by setting up our API connections to OpenAI for LLM queries and Contextual AI for reranking.

Sign up for a free trial of Contextual [here](https://app.contextual.ai/) to find `CONTEXTUAL_API_KEY`

For the reranker prompt generator and baseline model, we have used OpenAI's `GPT-4o-mini`, you can find your OpenAI API key [here](https://platform.openai.com/api-keys) or swap this section out to use the LLM of your choice.

In [ ]:
# Get API keys from Google Colab secrets
contextual_api_key = userdata.get("CONTEXTUAL_API_KEY")
openai_api_key = userdata.get("OPENAI_API_KEY")


# Initialize clients
llm_client = OpenAI(api_key=openai_api_key)
contextual_client = ContextualAI(api_key=contextual_api_key, base_url="https://api.contextual.ai/v1")

## Step (2) Next, we fetch all available MCP servers from the Pulse MCP API and format them as documents for reranking.

In [ ]:
# Fetch all MCP servers from Pulse MCP API
servers = []
offset = 0

while True:
    response = requests.get(
        "https://api.pulsemcp.com/v0beta/servers",
        params={"count_per_page": 5000, "offset": offset},
        headers={"User-Agent": "MCPCollector/1.0"}
    )

    data = response.json()
    batch = data["servers"]

    servers.extend(batch)

    if not data.get("next"):
        break

    offset += 5000

mcp_servers = [{
    "name": server["name"],
    "short_description": server["short_description"],
    "github_stars": server["github_stars"],
    "package_download_count": server["package_download_count"],
    "EXPERIMENTAL_ai_generated_description": server["EXPERIMENTAL_ai_generated_description"],
    "source_code_url": server.get("source_code_url"),
    "package_registry": server.get("package_registry"),
    "package_name": server.get("package_name"),
    "remotes": server.get("remotes", [])
} for server in servers]

print(f"Loaded {len(mcp_servers)} MCP servers")

# Prepare documents and metadata for reranking
documents = []
metadata = []

for server in mcp_servers:
    doc_parts = [
        f"MCP Server: {server['name']}",
        f"Description: {server['short_description']}",
    ]

    # # This would be nice but adding the ai generated description will exceed token count
    # if server['EXPERIMENTAL_ai_generated_description']:
    #     doc_parts.append(f"AI Description: {server['EXPERIMENTAL_ai_generated_description']}")

    documents.append("\n".join(doc_parts))
    metadata.append(f"Name: {server['name']}, Stars: {server['github_stars']}, Downloads: {server['package_download_count']}, Downloads: {server['remotes']}")

print(f"Prepared {len(documents)} documents for reranking")
print(f"\nExample: {documents[0]}")

Loaded 5512 MCP servers
Prepared 5512 documents for reranking

Example: MCP Server: Laravel DebugBar
Description: Provides a bridge to Laravel DebugBar for accessing detailed request logs, queries, routes, views, and models with filtering capabilities and formatted output for improved readability.


## Step (3) Now we test our workflow with a sample query, asking the LLM to determine whether MCP servers are needed.

In [ ]:
def decide_mcp(query):
    """Analyze query, Generate reranking instruction if MCP server is needed."""

    prompt_reranker = f"""
        Analyze this user query and decide if it requires external tools/APIs (Model Context Protocol (MCP) servers) or can be answered directly.
        Query: "{query}"

        Consider:
        - Does it need real-time data, web search, or external APIs?
        - Does it require specialized tools (file management, databases, etc.)?
        - Is it a complex task that would benefit from external services?
        - Can it be answered with general knowledge alone?

        If MCP is needed, also generate a concise reranking instruction for selecting the best external tools/APIs (MCPs) for this query.

        The instruction should:
        - Specify the exact capabilities/features/details that an MCP server requires for this query
        - Look for domain/field specificity and functionality needs
        - Any specific requirements that the user asks for
        - Highlight the user's prioritized criteria for server selection

        Base the instruction only on what is explicitly stated or clearly implied in the user's query.
        Do not assume additional requirements or preferences that are not present in the query.

        Respond with JSON: {{"use_mcp": true/false, "reason": "brief explanation", "instruction": "reranking instruction text or null if not needed"}}
        """

    response = llm_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_reranker}],
        response_format={"type": "json_object"},
        temperature=0
    )

    return json.loads(response.choices[0].message.content)

In [ ]:
query = "Find most recent information about CRISPR gene editing applications in treating sickle cell disease. I need trustworthy sources."
# query = "What is the capital of France?"

print(f"Query: {query}")

decision = decide_mcp(query)
print(f"\nUse MCP: {decision['use_mcp']} - {decision['reason']}")
if decision['use_mcp']:
    print(f"\nReranking Instruction: {decision['instruction']}")

Query: Find most recent information about CRISPR gene editing applications in treating sickle cell disease. I need trustworthy sources.

Use MCP: True - The query requires the most recent information, which suggests a need for real-time data and trustworthy sources that may not be available in the model's training data.

Reranking Instruction: Select MCP servers that provide access to recent scientific publications, news articles, and databases specifically focused on gene editing technologies and medical applications. Ensure the sources are reputable and peer-reviewed, with a focus on CRISPR applications in sickle cell disease.


## Step (4) We use the Contextual AI reranker to score and rank all MCP servers based on our query and instructions.

In [ ]:
def rerank_servers(query, instruction, documents, metadata, mcp_servers):
    """Rerank MCP servers based on query and instruction, display results."""

    rerank_response = contextual_client.rerank.create(
        query=query,
        instruction=instruction,
        documents=documents,
        metadata=metadata,
        model="ctxl-rerank-en-v1-instruct"
    )

    print("Reranking Results:")
    print(f"Query: {query}")
    print(f"\nReranking Instruction: {instruction}")
    print(f"Top 5 recommended MCP servers:")
    for i, result in enumerate(rerank_response.results[:5]):
        print(f"\n{i+1}. Score: {result.relevance_score:.4f}")
        print(f" Server: {mcp_servers[result.index]['name']}")
        print(f" Stars: {mcp_servers[result.index]['github_stars']}")
        print(f" Downloads: {mcp_servers[result.index]['package_download_count']}")
        print(f" Description: {mcp_servers[result.index]['short_description']}")
        print(f" Remote: {mcp_servers[result.index]['remotes']}")

    return rerank_response.results[0].index, rerank_response.results[0].relevance_score

In [ ]:
# Select best MCP server using reranker
if decision['use_mcp']:
    best_server_index, relevance_score = rerank_servers(query, decision['instruction'], documents, metadata, mcp_servers)
    selected_server = mcp_servers[best_server_index]
else:
    print("No server is needed.")

Reranking Results:
Query: Find most recent information about CRISPR gene editing applications in treating sickle cell disease. I need trustworthy sources.

Reranking Instruction: Select MCP servers that provide access to recent scientific publications, news articles, and databases specifically focused on gene editing technologies and medical applications. Ensure the sources are reputable and peer-reviewed, with a focus on CRISPR applications in sickle cell disease.
Top 5 recommended MCP servers:

1. Score: 0.6540
 Server: Gemini with Web Search
 Stars: 4
 Downloads: None
 Description: Integrates Google's Gemini model with real-time web search and YouTube data to enhance AI responses with up-to-date information from the internet.
 Remote: []

2. Score: 0.6301
 Server: PubMed Medical Literature Research
 Stars: 1
 Downloads: None
 Description: Enables search, retrieval, and analysis of academic medical literature through the NCBI PubMed database with formatted citations and researcher pu

# Comparison to baseline

In [ ]:
# Truncate documents to fit within OpenAI context length
max_tokens = 128000 - 2000
max_chars = (max_tokens // len(documents)) * 4
docs_GPT = [doc[:max_chars] for doc in documents]

In [ ]:
def baseline_approach(q, docs_GPT):
    """Get baseline MCP server recommendations using simple LLM prompt."""

    prompt_baseline = f"""
        You are given a user query and a list of MCP servers. Query: {q} MCP Servers: {docs_GPT}.

        Return the top 5 most relevant MCP server from the list to answer this query.
        Provide their server names.

        If no MCP server is relevant or needed, respond with "None".
        """

    response_baseline = llm_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_baseline.replace("{query}", q)}],
        temperature=0,
    )

    return response_baseline.choices[0].message.content

In [ ]:
queries = [
    # "Help me manage GitHub repositories with automated testing and CI/CD workflows.",
    # "Help me automate my g-mail responses while I'm out of office.",
    # "Retrieve latest NBA game scores with official league statistics.",
    # "Get real-time trading info for APPL. Do some AI-driven financial analysis on it.",
    # "Show trending opinions and discussions about climate change. Only consider non-US platforms.",
    # "I have many complex enterprise documents. I want to build an agent that lets me interact with and query these docs through natural conversation."
    "I want to send an email or a text or call someone via MCP, and I want the server to be remote and have high user rating"
]

for q in queries:
    print(f"\n--- Query: {q} ---")
    ##################### Reranker approach #####################
    decision = decide_mcp(q)
    print(f"\nUse MCP: {decision['use_mcp']} - {decision['reason']}")
    if decision['use_mcp']:
        print(f"\nReranking Instruction: {decision['instruction']}")
        rerank_servers(q, decision['instruction'], documents, metadata, mcp_servers)

    ##################### Baseline approach #####################
    print("\nBaseline top 5 MCP servers:")
    print(baseline_approach(q, docs_GPT))


--- Query: I want to send an email or a text or call someone via MCP, and I want the server to be remote and have high user rating ---

Use MCP: True - The user is looking for a remote server with high user ratings to send an email, text, or make a call, which requires real-time data and external APIs.

Reranking Instruction: Select MCP servers that offer capabilities for sending emails, texts, and making calls. Ensure the servers are remote and have high user ratings. Prioritize those with a user-friendly interface and reliable performance in communication services.
Reranking Results:
Query: I want to send an email or a text or call someone via MCP, and I want the server to be remote and have high user rating

Reranking Instruction: Select MCP servers that offer capabilities for sending emails, texts, and making calls. Ensure the servers are remote and have high user ratings. Prioritize those with a user-friendly interface and reliable performance in communication services.
Top 5 rec

## Easy queries:

For queries that are straightforward, both the reranker and baseline LLM give similar results.

---

**Query 1: Help me manage GitHub repositories with automated testing and CI/CD workflows.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **GitHub**¹ | GitHub |
| 2 | **GitHub**² | GitHub Actions |
| 3 | **GitHub (via OAuth)**³ | GitHub PR Helper |
| 4 | **GitHub**⁴ | GitHub Projects |
| 5 | **GitHub**⁵ | GitHub Repository Browser |

¹ Integrates with GitHub's API to enable repository management, file operations, issue tracking, pull request handling, and workflow automation with configurable security controls for development workflows.  
² Integrates with GitHub's API to enable repository management, code manipulation, and issue tracking for streamlined development workflows.  
³ Integrates with GitHub's OAuth system to enable secure access and management of repositories, issues, and code for automated development workflows and analysis.  
⁴ Integrates with GitHub using Octokit to enable repository management, issue tracking, and code analysis.  
⁵ Integrates with GitHub's API to enable repository management, code manipulation, issue tracking, and pull request handling for streamlined development workflows.

---

**Query 2: Help me automate my g-mail responses while I'm out of office.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **Gmail**¹ | Gmail |
| 2 | **Gmail**² | Google Workspace |
| 3 | **Google Workspace Automation**³ | Email Sender |
| 4 | **Gmail**⁴ | Outlook Email Processor |
| 5 | **Gmail**⁵ | Resend Email |

¹ Integrates with Gmail's API to enable email operations including sending, receiving, and managing emails for automation and workflow integration.  
² Integrates with Gmail APIs to enable reading, sending, and managing emails across multiple accounts, supporting automated email processing and workflow integration.  
³ Integrates with Google Workspace services through Google Apps Script Web Apps to automate Gmail, Calendar, Drive, Docs, Sheets, Slides, and Forms operations including email management, event scheduling, file handling, document creation, and form building with weather data retrieval and presentation generation capabilities.  
⁴ Integrates with Gmail's API to enable email management tasks like sending, reading, drafting, and organizing messages.  
⁵ Integrates with Gmail to enable natural language-based email management, including searching, reading, deleting, and sending emails with attachment support and conversation threading.

---

**Query 3: Retrieve latest NBA game scores with official league statistics.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **NBA Stats**¹ | MCP Server: NBA Stats |
| 2 | **Web Search (Brave)**² | MCP Server: NBA Player Stats |
| 3 | **NBA Stats**³ | MCP Server: NBA Stats |
| 4 | **MCP Advisor**⁴ | MCP Server: NBA Player Stats |
| 5 | **WebMCP**⁵ | MCP Server: NBA Stats |

¹ Provides real-time NBA game data including scores, player statistics, and play-by-play information through the nba_api package for accessing current basketball information not in Claude's training data.  
² Integrates Brave Search API for real-time web information retrieval, dynamically categorizing queries to proactively fetch current events, weather, sports scores, and time-sensitive data with robust error handling.  
³ Provides a Python-based interface to NBA statistics and live game data, offering tools for accessing game information, player statistics, team data, and league standings without requiring knowledge of the underlying API structure.  
⁴ Discovery and recommendation service that helps find and understand available MCP services based on natural language queries, supporting multiple search backends for exploring servers by semantic similarity.  
⁵ Turn any website into an MCP server with customizable tools.

## Queries with specific requirements

For more complicated queries with specific requirements from the user, the reranker is able to prioritize the user's preferences.

---

**Query 1: Get real-time trading info for APPL. Do some AI-driven financial analysis on it.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **XTQuant AI**¹ | Yahoo Finance |
| 2 | **Stockflow (Yahoo Finance)**² | AlphaVantage Trader |
| 3 | **Yahoo Finance**³ | CoinGecko |
| 4 | **Yahoo Finance**⁴ | Stock Market (Alpha Vantage) |
| 5 | **Alpha Vantage**⁵ | Zerodha Kite |

¹ Lightweight Python server for quantitative trading using XTQuant API, enabling real-time stock market interactions and AI-driven financial analysis strategies.  
² Integrates with the Yahoo Finance API to provide real-time stock market data and analysis tools for financial research and trading strategies.  
³ Provides real-time financial data and visualization tools through Yahoo Finance API for stock prices, company information, and market analysis with interactive charts and dashboards.  
⁴ Provides real-time financial data from Yahoo Finance through specialized tools for retrieving stock information, market trends, and news for investment research and analysis.  
⁵ Provides real-time financial data retrieval from Alpha Vantage, enabling developers and analysts to access comprehensive stock prices, market indicators, forex rates, and cryptocurrency information with instant, structured results.

---

**Query 2: Show trending opinions and discussions about climate change. Only consider non-US platforms.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **Chinese Trends Hub**¹ | ClimateTriage |
| 2 | **Hotnews (Chinese Social)**² | Twitter Search |
| 3 | **Beyond Social (Farcaster)**³ | Reddit |
| 4 | **PulseMCP**⁴ | GNews |
| 5 | **X (Twitter)**⁵ | Newsdata |

¹ Provides real-time access to trending topics and content from major Chinese platforms including Weibo, Zhihu, Douyin, Bilibili, Douban, Toutiao, and 36kr through separate tools with temporary caching for improved performance.  
² Aggregates real-time trending topics from major Chinese social platforms and news sites.  
³ Integrates with social platforms like Farcaster to provide standardized access to user profiles, content search, thread analysis, and trending topics through flexible stdio and HTTP/SSE transports.  
⁴ Integrates with PulseMCP to enable querying and retrieval of MCP server and integration data for ecosystem analysis and service recommendations.  
⁵ Integrates with X's API to enable tweet retrieval, user interactions, and social media analysis capabilities.


---

**Query 3: I need to process large-scale datasets from my PostgreSQL database efficiently. Find an easy way to do it.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **Ollama PostgreSQL Data Analysis**¹ | PostgreSQL |
| 2 | **DBHub (Universal Database Gateway)**² | PostgreSQL |
| 3 | **PostgreSQL Database Manager**³ | PostgreSQL |
| 4 | **Supabase PostgreSQL**⁴ | PostgreSQL |
| 5 | **PostgreSQL Database Manager**⁵ | PostgreSQL |

¹ Integrates natural language querying with PostgreSQL databases to enable AI-interpreted data analysis without manual SQL writing.  
² Provides a universal database gateway for connecting to PostgreSQL, MySQL, SQLite, and DuckDB, enabling table browsing, schema inspection, and read-only SQL queries with built-in safety checks.  
³ Integrates with PostgreSQL databases to enable schema management, data migration, performance monitoring, and security configuration through direct database operations without requiring separate management tools.  
⁴ Integrates with Supabase PostgreSQL databases, exposing table schemas as resources and providing tools for data analysis, including read-only SQL queries, table statistics, and relationship discovery.  
⁵ Integrates with PostgreSQL to provide database analysis, optimization, and troubleshooting tools for streamlined management and performance tuning.

---

**Query 4: I have many complex enterprise documents. I want to build an agent that lets me interact with and query these docs through natural conversation.**

| Rank | Reranker | Baseline |
|------|----------|---------|
| 1 | **Contextual AI**¹ | Document Operations (Word, Excel, PDF) |
| 2 | **Lokka (Microsoft Graph)**² | PDF Reader |
| 3 | **DocuMCP (RAG Documentation Server)**³ | Documentation Manager |
| 4 | **Glean**⁴ | Markdown to PDF |
| 5 | **Kollektiv Document Management**⁵ | Google Docs |

¹ Integrates with Contextual AI's platform to provide enterprise search, multi-agent routing with automatic reranking, and intelligent document navigation that leverages hierarchical structure analysis for conversational access to knowledge bases and complex documents.  
² Provides a bridge between Microsoft Graph API and natural language interaction, enabling conversational management of Microsoft 365 tenants without complex API calls.  
³ RAG-enabled documentation server that integrates with vector databases to provide semantic search capabilities for code, documentation, and diagrams without data leaving your environment.  
⁴ Integrates Glean's search and chat APIs to enable natural language interactions with organizational knowledge bases and chatbots.  
⁵ Enables AI to search and analyze user-uploaded documents through Kollektiv's document management system with secure OAuth authentication and natural language querying capabilities.


Cost estimate:
*   Contextual AI Reranker (with full MCP docs): ~$0.035/query

Includes 0.035 for reranking + ~$0.0001 for OpenAI instruction generation

*   OpenAI Baseline: ~$0.017/query

